In [3]:
# !wget https://huggingface.co/datasets/mesolitica/chatgpt-malaysian-qa-choice/resolve/main/qa-majalahsains.jsonl

In [12]:
import json
import random
import re

def clean(string):
    string = re.sub('[^A-Za-z ]+', ' ', string.lower())
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

def overlap(string1, string2):
    l = set([w for w in clean(string1).split() if len(w) > 2])
    r = set([w for w in clean(string2).split() if len(w) > 2])
    return len(l & r) / len(l)

In [5]:
data = []
with open('qa-majalahsains.jsonl') as fopen:
    for l in fopen:
        data.append(json.loads(l))

In [6]:
data[0]

{'paragraph': 'Di sini kami tunjukkan satu video aktiviti penyelidikan yang dijalankan di Institute of Microengineering & Nanoelectronics (IMEN), yang terletak di Universiti Kebangsaan Malaysia.\n\nDi sini kami tunjukkan satu video aktiviti penyelidikan yang dijalankan di Institute of Microengineering & Nanoelectronics (IMEN), yang terletak di Universiti Kebangsaan Malaysia.\n\nIMEN yang diketuai oleh Profesor Dato’ Dr. Burhanuddin Yeop Majlis selaku Pengarah ditubuhkan pada bulan November 2002. Institut ini menumpukan penyelidikan multidisiplin dalam bidang Kejuruteraan Mikro, Nanoteknologi, Nanoelektronik dan menerima anugerah Pusat Kecemerlangan Pengajian Tinggi (HiCOE), pada tahun 2015 di dalam bidang ini.\n\nIMEN yang diketuai oleh Profesor Dato’ Dr. Burhanuddin Yeop Majlis selaku Pengarah ditubuhkan pada bulan November 2002. Institut ini menumpukan penyelidikan multidisiplin dalam bidang Kejuruteraan Mikro, Nanoteknologi, Nanoelektronik dan menerima anugerah Pusat Kecemerlangan P

In [7]:
len(data)

2321

In [8]:
indices = {i for i in range(len(data))}

In [13]:
from tqdm import tqdm

pairs = []
for i in tqdm(range(len(data))):
    query = data[i]['paragraph']
    
    for s in data[i]['qa']['qa']:
        pos = [query]
        sampled = random.sample(list(indices - {i}), 100)
        neg = []
        for k in sampled:
            try:
                overlapped = overlap(query, data[k]['paragraph'])
                if overlapped < 0.1:
                    neg.append(data[k]['paragraph'])
                if len(neg) >= 5:
                    break
            except:
                pass
            
        pairs.append({
            'query': s['question'],
            'pos': pos,
            'neg': neg
        })

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2321/2321 [02:45<00:00, 14.05it/s]


In [14]:
pairs[0]

{'query': 'Apakah nama institusi yang melakukan aktiviti penyelidikan?',
 'pos': ['Di sini kami tunjukkan satu video aktiviti penyelidikan yang dijalankan di Institute of Microengineering & Nanoelectronics (IMEN), yang terletak di Universiti Kebangsaan Malaysia.\n\nDi sini kami tunjukkan satu video aktiviti penyelidikan yang dijalankan di Institute of Microengineering & Nanoelectronics (IMEN), yang terletak di Universiti Kebangsaan Malaysia.\n\nIMEN yang diketuai oleh Profesor Dato’ Dr. Burhanuddin Yeop Majlis selaku Pengarah ditubuhkan pada bulan November 2002. Institut ini menumpukan penyelidikan multidisiplin dalam bidang Kejuruteraan Mikro, Nanoteknologi, Nanoelektronik dan menerima anugerah Pusat Kecemerlangan Pengajian Tinggi (HiCOE), pada tahun 2015 di dalam bidang ini.\n\nIMEN yang diketuai oleh Profesor Dato’ Dr. Burhanuddin Yeop Majlis selaku Pengarah ditubuhkan pada bulan November 2002. Institut ini menumpukan penyelidikan multidisiplin dalam bidang Kejuruteraan Mikro, Nanot

In [15]:
with open('qa-majalahsains-pair.jsonl', 'w') as fopen:
    for l in pairs:
        if not len(l['pos']) or not len(l['neg']):
            continue
        fopen.write(f'{json.dumps(l)}\n')

In [16]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='qa-majalahsains-pair.jsonl',
    path_in_repo='qa-majalahsains-pair.jsonl',
    repo_id='mesolitica/instructions-pair-mining',
    repo_type='dataset',
)

qa-majalahsains-pair.jsonl:   0%|          | 0.00/178M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/instructions-pair-mining/commit/c271cd49c36038b4e46feffd272cd3570d96e99f', commit_message='Upload qa-majalahsains-pair.jsonl with huggingface_hub', commit_description='', oid='c271cd49c36038b4e46feffd272cd3570d96e99f', pr_url=None, pr_revision=None, pr_num=None)